# Notebook pour l'implémentation d'une première pipeline

Read et write AAP pour la V1 du 31/3/2025 

In [ ]:

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "SCAN PARAGRAPHS & TABLES TOP DOWN IN A DOCX DOCUMENT" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def iter_block_items(parent):
    """
    Yield each paragraph and table child within *parent*, in document
    order. Each returned value is an instance of either Table or
    Paragraph. *parent* would most commonly be a reference to a main
    Document object, but also works for a _Cell object, which itself can
    contain paragraphs and tables.
    """
    import docx
    from docx.document import Document
    from docx.oxml.table import CT_Tbl
    from docx.oxml.text.paragraph import CT_P
    from docx.table import _Cell, Table
    from docx.text.paragraph import Paragraph

    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "SCAN PARAGRAPHS & TABLES TOP DOWN IN A DOCX DOCUMENT" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "ONE OF THE WORDS IS IN THE PARAGRAPH" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def OneOfTheWords_Is_InTheParagraph (TheText, list_of_Words_OK, list_of_Words_KO):
    """
    This function verifies if one of the words of a list of words is in a paragraph
        
    Args:
        docpara : the paragraph in which we verify
        list_of_Words_OK : List of words that we want to check if they are present in the paragraph
        list_of_Words_KO : List of words that indicate wrong interpretation of the Words OK
            in other terms, if we find a word of list_of_Words_OK in the paragraph but we also 
            find there a word of list_of_Words_KO, it disqualifies the 1st finding and we consider no presence of the word in the paragraph
            e.g. : we want to find the word meaning "maximum" so we look for "MAX" (OK list) because maximum is often written "max."
            we find it, but find also "MAXIMIZE" (KO list), in this case MAX does not means "MAXIMUM" but it is part of "MAXIMIZE" which is wrong for our quest
            
    Returns:
        The function returns True if a word from list_of_Words_OK is found 
        and no word from list_of_Words_KO is found
         Else it returns False
    """
    FlagWord_OK = False # by default, we consider no word found in the paragraph
        #============== 1 - TREATMENT OF "LIST_OF_WORDS_OK" ================================================    
    for Theword in list_of_Words_OK: # We look for words from the list list_of_Words_OK
        #if the word in lowercase is in the text in lowercase, we have found one matching word
        if re.search(Theword.lower(), TheText.lower(), flags=0)!= None:
            FlagWord_OK = True

        #============== 2 - TREATMENT OF "LIST_OF_WORDS_KO" ================================================    
    for Theword in list_of_Words_KO: # Now we look for words from the list list_of_Words_KO
        #if the word in lowercase is in the text in lowercase, we have found one matching word
        if re.search(Theword.lower(), TheText.lower(), flags=0)!= None:
            FlagWord_OK = False # the Word of list_of_Words_KO disqualifies the word of list_of_Words_OK
            #if the keyword in lowercase is in the text in lowercase, we have found one matching word
    return FlagWord_OK # return True if found or False if not found

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "ONE OF THE WORDS IS IN THE PARAGRAPH" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "INSERT TEXT IN ONE PARAGRAPH IN FULL TEXT (NO TABLE)" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def Insert_Text_Paragraph (block_item, TextStart, TextEnd):
    """
    This function inserts text into a paragraph of Word docx at the beginning and at the end of the paragraph
    This function works for paragraphs in full text (i.e. not inside tables)
    To insert text inside cells of tables, another code is required
    This code allows to insert the text in the paragraph using "replace" function which is the only way to do it
    without loosing the initial look & feel of the texte (size, font, color,..)
    because any other way of changing the text of a paragraph in docx Word will unfortunately loose all of that

    Args:
        block_item : the paragraph in which we insert the text
        TextStart : the text to be inserted at the beginning of the paragraph
        TextEnd : the text to be inserted at the end of the paragraph
            
    Returns:
        The function returns nothing
        but modifies the paragraph by adding text
    """
    if block_item.runs == []: # if the paragraph has no run
        block_item.text = TextStart + block_item.text + TextEnd # we manage at text level
    else: # if the paragraph has at least 1 run, we manage at run level
        # insert the start text
        block_item.runs[0].text = block_item.runs[0].text.replace("", TextStart,1) 
        # then insert the end text
        NbRuns = block_item.runs.__len__()
        block_item.runs[NbRuns-1].text = block_item.runs[NbRuns-1].text.replace(block_item.runs[NbRuns-1].text, block_item.runs[NbRuns-1].text + TextEnd,1)

    return
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "INSERT TEXT IN ONE PARAGRAPH IN FULL TEXT (NO TABLE)" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "DELETE TEXT IN ONE PARAGRAPH IN FULL TEXT (NO TABLE)" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def Delete_Text_Paragraph (block_item, Text_to_delete):
    """
    This function deletes text in a paragraph of Word docx
    This function works for paragraphs in full text (i.e. not inside tables)
    To delete text inside cells of tables, another code is required
    This code allows to delete the text in the paragraph using "replace" function which is the only way to do it
    without loosing the initial look & feel of the texte (size, font, color,..)
    because any other way of changing the text of a paragraph in docx Word will unfortunately loose all of that

    Args:
        block_item : the paragraph in which we insert the tags
        Text_to_delete : the text to be deleted in the paragraph
            
    Returns:
        The function returns nothing
        but modifies the paragraph by deleting text
    """
    Text_to_delete2 =""
    if block_item.runs == []: # if the paragraph has no run
        block_item.text = block_item.text.replace(Text_to_delete, "") # suppress the Text_to_delete but loose the initial look & feel (format) of the paragraph)

    else: # if the paragraph has at least 1 run, we manage at run level
        # we have to create Text_to_delete2 because re.search will not work with simple ? or > or < or /
        if Text_to_delete =="??": 
            Text_to_delete2 = r'\?\?'
        if Text_to_delete =="<>":
            Text_to_delete2 = r'\<\>'
        if Text_to_delete =="</>":
            Text_to_delete2 = r'\<\/\>'
        
        NbRuns = block_item.runs.__len__()
        for i in range(NbRuns):  # Loop through all runs in the paragraph
            MyRun = block_item.runs[i]
            if Text_to_delete2 !="": # if it is "??" or "<>"" or "</>", we use Text_to_delete2
                if re.search(Text_to_delete2, MyRun.text, flags=0)!= None :
                    MyRun.text = MyRun.text.replace(Text_to_delete, '',1) 
            else: # if it is NOT "??" or "<>"" or "</>", use Text_to_delete
                if re.search(Text_to_delete, MyRun.text, flags=0)!= None :
                    MyRun.text = MyRun.text.replace(Text_to_delete, '',1) 
        if Text_to_delete2 !="" and re.search(Text_to_delete2, block_item.text, flags=0)!= None :# if after run treatment, the text to delete not deleted (runs cut le texte to delete in 2)
            # Try to save the former format of paragraph by saving the format of the last run
            NbRuns = block_item.runs.__len__()
            MyFontName = block_item.runs[NbRuns-1].font.name
            MyFontSize = block_item.runs[NbRuns-1].font.size
            MyFontBold = block_item.runs[NbRuns-1].font.bold
            MyFontItalic = block_item.runs[NbRuns-1].font.italic
            MyFontUnderline = block_item.runs[NbRuns-1].font.underline
            MyFontColor = block_item.runs[NbRuns-1].font.color.rgb
            block_item.text = block_item.text.replace(Text_to_delete, "") # suppress the Text_to_delete but loose the initial look & feel (format) of the paragraph)
            # Try to re establish the former style
            NbRuns = block_item.runs.__len__()
            for i in range(NbRuns):  # Loop through all runs in the paragraph
                MyRun = block_item.runs[i]
                MyRun.font.name = MyFontName
                MyRun.font.size = MyFontSize
                MyRun.font.bold = MyFontBold
                MyRun.font.italic = MyFontItalic
                MyRun.font.underline = MyFontUnderline
                MyRun.font.color.rgb = MyFontColor

        if Text_to_delete2 =="" and re.search(Text_to_delete, block_item.text, flags=0)!= None :# if after run treatment, the text to delete not deleted (runs cut le texte to delete in 2)
            # Try to save the former format of paragraph by saving the format of the last run
            NbRuns = block_item.runs.__len__()
            MyFontName = block_item.runs[NbRuns-1].font.name
            MyFontSize = block_item.runs[NbRuns-1].font.size
            MyFontBold = block_item.runs[NbRuns-1].font.bold
            MyFontItalic = block_item.runs[NbRuns-1].font.italic
            MyFontUnderline = block_item.runs[NbRuns-1].font.underline
            MyFontColor = block_item.runs[NbRuns-1].font.color.rgb
            block_item.text = block_item.text.replace(Text_to_delete, "") # suppress the Text_to_delete but loose the initial look & feel (format) of the paragraph)
            # Try to re establish the former style
            NbRuns = block_item.runs.__len__()
            for i in range(NbRuns):  # Loop through all runs in the paragraph
                MyRun = block_item.runs[i]
                MyRun.font.name = MyFontName
                MyRun.font.size = MyFontSize
                MyRun.font.bold = MyFontBold
                MyRun.font.italic = MyFontItalic
                MyRun.font.underline = MyFontUnderline
                MyRun.font.color.rgb = MyFontColor


    return
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "DELETE TEXT IN ONE PARAGRAPH IN FULL TEXT (NO TABLE)" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "INSERT TEXT IN ONE CELL OF A TABLE" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def Insert_Text_Cell (tableCell, TextStart, TextEnd):
    """
    This function inserts text in a cell of a table of Word docx at the beginning and at the end of the text
    This function works only for paragraphs inside tables
    To insert text in paragraphs in full text, another code is required
    This code allows to insert the text into the paragraph using "replace" function which is the only way to do it
    without loosing the initial look & feel of the texte (size, font, color,..)
    because any other way of changing the text of a paragraph in docx Word will unfortunately loose all of that

    Args:
        tableCell : the cell of a table in which we insert the text
        TextStart : the tag to be inserted at the beginning of the paragraph
        TextEnd : the tag to be inserted at the end of the paragraph
            
    Returns:
        The function returns nothing
        but modifies the paragraph in the table cell by adding text
    """
    # scan the paragraphs of the cell and insert the text
    ListOfRuns = []
    for paragCell in tableCell.paragraphs:
        ListOfRuns.extend(paragCell.runs)
    NbRuns = len(ListOfRuns)
    
    if NbRuns == 0: # if the cell has no run
        tableCell.text = tableCell.text.replace(tableCell.text, TextStart + tableCell.text + TextEnd) # insert the TextStart and TextEnd
    else: # if there is at least 1 run
        # insert the start text
        ListOfRuns[0].text = ListOfRuns[0].text.replace("", TextStart,1) 
        # insert the end text
        ListOfRuns[NbRuns-1].text = ListOfRuns[NbRuns-1].text.replace(ListOfRuns[NbRuns-1].text, ListOfRuns[NbRuns-1].text + TextEnd,1)
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "INSERT TEXT IN ONE CELL OF A TABLE" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "DELETE TEXT IN ONE CELL OF A TABLE" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def Delete_Text_Cell (tableCell, Text_to_delete):
    """
    This function deletes text in a cell of a table of Word docx
    This function works only for paragraphs inside tables
    To delete text in paragraphs in full text, another code is required
    This code allows to delete the text of the paragraph using "replace" function which is the only way to do it
    without loosing the initial look & feel of the texte (size, font, color,..)
    because any other way of changing the text of a paragraph in docx Word will unfortunately loose all of that

    Args:
        tableCell : the cell of a table in which we delete the text
        Text_to_delete : the text to be deleted in the cell
            
    Returns:
        The function returns nothing
        but modifies the paragraph in the table cell by deleting text
    """        
    
    Text_to_delete2 =""
    # scan the paragraphs of the cell
    ListOfRuns = []
    for paragCell in tableCell.paragraphs:
        ListOfRuns.extend(paragCell.runs)
    NbRuns = len(ListOfRuns)

    if NbRuns == 0: # if the cell has no run
        tableCell.text = tableCell.text.replace(Text_to_delete, "") # replace by ''

    else: # if the paragraph has at least 1 run, we manage at run level
        # we have to create Text_to_delete2 because re.search will not work with simple ? or > or < or /
        if Text_to_delete =="??": 
            Text_to_delete2 = r'\?\?'
        if Text_to_delete =="<>":
            Text_to_delete2 = r'\<\>'
        if Text_to_delete =="</>":
            Text_to_delete2 = r'\<\/\>'

        for i in range(NbRuns):  # Loop through all runs in the paragraph
            MyRun = ListOfRuns [i]
            if Text_to_delete2 !="": # if it is "??" or "<>"" or "</>", we use Text_to_delete2
                if re.search(Text_to_delete2, MyRun.text, flags=0)!= None :
                    MyRun.text = MyRun.text.replace(Text_to_delete, '',1) 
            else: # if it is NOT "??" or "<>"" or "</>", use Text_to_delete
                if re.search(Text_to_delete, MyRun.text, flags=0)!= None :
                    MyRun.text = MyRun.text.replace(Text_to_delete, '',1) 


        if Text_to_delete2 !="" and re.search(Text_to_delete2, tableCell.text, flags=0)!= None :# if after run treatment, the text to delete not deleted (runs cut le texte to delete in 2)
            # Try to save the former format of paragraph by saving the format of the last run
            ListOfRuns = []
            for paragCell in tableCell.paragraphs:
                ListOfRuns.extend(paragCell.runs)
            NbRuns = len(ListOfRuns)
            MyFontName = ListOfRuns [NbRuns-1].font.name
            MyFontSize = ListOfRuns [NbRuns-1].font.size
            MyFontBold = ListOfRuns [NbRuns-1].font.bold
            MyFontItalic = ListOfRuns [NbRuns-1].font.italic
            MyFontUnderline = ListOfRuns [NbRuns-1].font.underline
            MyFontColor = ListOfRuns [NbRuns-1].font.color.rgb
            tableCell.text = tableCell.text.replace(Text_to_delete, "") # suppress the Text_to_delete but loose the initial look & feel (format) of the paragraph)
            # Try to re establish the former style
            ListOfRuns = []
            for paragCell in tableCell.paragraphs:
                ListOfRuns.extend(paragCell.runs)
            NbRuns = len(ListOfRuns)
            for i in range(NbRuns):  # Loop through all runs in the paragraph
                MyRun = ListOfRuns[i]
                MyRun.font.name = MyFontName
                MyRun.font.size = MyFontSize
                MyRun.font.bold = MyFontBold
                MyRun.font.italic = MyFontItalic
                MyRun.font.underline = MyFontUnderline
                MyRun.font.color.rgb = MyFontColor

        if Text_to_delete2 =="" and re.search(Text_to_delete, tableCell.text, flags=0)!= None :# if after run treatment, the text to delete not deleted (runs cut le texte to delete in 2)
            # Try to save the former format of paragraph by saving the format of the last run
            ListOfRuns = []
            for paragCell in tableCell.paragraphs:
                ListOfRuns.extend(paragCell.runs)
            NbRuns = len(ListOfRuns)
            MyFontName = ListOfRuns [NbRuns-1].font.name
            MyFontSize = ListOfRuns [NbRuns-1].font.size
            MyFontBold = ListOfRuns [NbRuns-1].font.bold
            MyFontItalic = ListOfRuns [NbRuns-1].font.italic
            MyFontUnderline = ListOfRuns [NbRuns-1].font.underline
            MyFontColor = ListOfRuns [NbRuns-1].font.color.rgb
            tableCell.text = tableCell.text.replace(Text_to_delete, "") # suppress the Text_to_delete but loose the initial look & feel (format) of the paragraph)
            # Try to re establish the former style
            ListOfRuns = []
            for paragCell in tableCell.paragraphs:
                ListOfRuns.extend(paragCell.runs)
            NbRuns = len(ListOfRuns)
            for i in range(NbRuns):  # Loop through all runs in the paragraph
                MyRun = ListOfRuns[i]
                MyRun.font.name = MyFontName
                MyRun.font.size = MyFontSize
                MyRun.font.bold = MyFontBold
                MyRun.font.italic = MyFontItalic
                MyRun.font.underline = MyFontUnderline
                MyRun.font.color.rgb = MyFontColor
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "DELETE TEXT IN ONE CELL OF A TABLE" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@ "READ QUESTIONS AND SIZE ANSWER REQUIREMENTS IN NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def Read_Questions_in_docx ( PathFolderSource, PathForOutputsAndLogs, list_of_SizeWords_OK, list_of_SizeWords_KO, TagQStart = "<>", TagQEnd = "</>" ):
    """
    CONTEXT:
    Uses python-docx 1.1.2 to manipulate Word documents : .docx only but not .doc. You need first to type "pip install python-docx" in your terminal
    Read the questions inside files contained in a folder with .docx extension and which are AAP ("Appel A Projet")
    AAP = document emitted by a donor describing the conditions under which it will grant funds to NGOs

    ACTIONS OF THE CODE
    Finds questions inside the AAP document and finds also information about 
    the size of answer required by the donor, if indicated (not always required)
    (e.g. : number max or min of words, characters, lines,..)
    Puts questions and size requirements into a dictionary with a Unique ID (UID) associated with each question
    Puts the UID into the AAP document at the right place


    ===========        How does it idenfiy the size of answer required ?   ==================
    It uses the lists "list_of_Answer_SizeWords" and "list_of_Exclus_SizeWords"
    In a paragraph, if a word of list_of_Answer_SizeWords is present and no word is present from list_of_Exclus_SizeWords,
    , then this paragraph includes the indication of size. 
    
    The indication of size is generally inside parentesis () in the same paragraph as the question, 
    In this case, it separates the question from the size requirement.
    The indication of size can also be in the following paragraph or a paragraph nearby
    If not found inside the question, it tries to find it around.
    It is common also that there is no indication of size required

    Args:
        PathFolderSource: Path to the folder containing the files to be read
        PathForOutputsAndLogs: Path to the folder containing the log file
        list_of_Answer_SizeWords and list_of_Exclus_SizeWords: used to identify in the texte the requirements 
            for the size of the answer given by the donor (see explainations above)
        TagQStart = "<>" Tag indicating the beginning of a Multi-paragraphs question (question with context below)
        TagQEnd = "</>" Tag indicating the end of a Multi-paragraphs question (question with context below)     

    Returns:
        The function returns a dictionary of the questions + size requirement if any in the following format :
            {
                UIDquestion1: [‘question1’, ‘Size answer1 (optional)‘,’‘,’‘,’’,''],
                UIDquestion2: [‘question2’, ‘Size answer2 (optional)',‘’,‘’,‘’,''],
                ...
            }
        The 4 empty fields in each question are for : "general context of AAP", "context for the question", "Qualification open or close question","answer given by IA"
        Context is not managed for the moment. The field "answer given by IA" will be filled by IA.
        It creates also a new version of the AAP document named "NameOfDocument-with_UID"
        It also logs errors in a file named "logs-IA_for_Asso.txt" in the folder "PathForOutputsAndLogs"
    """
    TheText = '' # Text on which we are working
    DictQuestions = {} #initialise the dictionnary of questions
    ListDict = [] #initialize list which is put into the dictionnary of questions
    EverythingOK = True # All the prerequisit chekings are OK if True
    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    FilesWithPath = []
    Multi_Paragraph = False # True if we are in a multi-paragraph,
    Go_DictionUID = False # True if it is OK to send the question to the dictionary and the AAP
    Text_Question = '' # The text of the question that we are going to put in the dictionary

    #======= A - FILES PREREQUISIT CHECK  ========

    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)

     #======= A.1 Only docx files in the folder ========
    for file in FilesWithPath:
        TheExtension = file [-4:]
        if TheExtension != "docx":
            EverythingOK = False
            MessageError = str(datetime.now()) + ' Error encountered when reading files : There should only be docx files in the folder'
            logging.error(MessageError)
            print (MessageError)

    for file in FilesWithPath:
        #======= B - OPEN THE DOCX FILE  ========
        NameOfWorkDocument = (file.split('/')[-1])[:-5]
        if EverythingOK and NameOfWorkDocument[len(NameOfWorkDocument)-9:] !="-with UID": #do not put UID where there are already UIDs
            try:
                docWork = Document(file)
            except IOError:
                MessageError = str(datetime.now()) + ' Error encountered when reading Word docx file ' + file
                logging.error(MessageError)
                print(MessageError)

            #============== C - RETRIEVE QUESTIONS AND SIZE REQUIREMENTS ================================================    
            for block_item in iter_block_items(docWork): # scan of working document top down (paragraphs @ tables)

            #============== 1 - RETREIVE QUESTIONS & SIZE IN A "FULL TEXT" PARAGRAPH (NOT IN A TABLE)================================================    

                #=======  1.2 - RETREIVE THE QUESTION FROM THE PARAGRAPH ========
                if isinstance(block_item, Paragraph): # treatment of a "full text" paragraph (not table) 

                    #=======  1.2.1 - TREATMENT IF NOT IN A MULTI-PARAGRAPH ========
                    if Multi_Paragraph == False:
                        if re.search(TagQStart, block_item.text, flags=0) == None : # if there is no TagStart of a multi-paragraph question in it

                            # If "?" in the simple paragraph, it is a question to be added
                            if re.search(r'\?', block_item.text, flags=0)!= None : # if there is a "?" in it
                                Text_Question = block_item.text # put the text into the Text of Question
                                Go_DictionUID = True # Go to send the question to the dictionary and the AAP
                            #else: # There is a tag start but no ?

                    
                        #=======  1.2.2 - TREATMENT IF IT IS THE START OF A MULTI-PARAGRAPH ========
                        else: # if a tag start of multi-paragraph is found = we enter in a Multi-Paragraph
                            Multi_Paragraph = True # it is the start of Multi Paragraph
                            Text_Question = Text_Question + block_item.text # put the text into the Text of Question
                            if re.search(TagQEnd, block_item.text, flags=0) != None : # if there is a TagEnd of a multi-paragraph question in it
                                # there is a Tag start and a Tag End in the same paragraph (should not happen theorically)
                                Multi_Paragraph = False # it is the end of Multi Pragraph
                                Go_DictionUID = True # Go to send the question to the dictionary and the AAP

                        #=======  1.2.3 - TREATMENT IF WE ARE ALREADY IN A MULTI-PARAGRAPH ========
                    else: # Multipragraph = True => we are already in a Multi-Paragraph
                        Text_Question = Text_Question + ' ' + block_item.text # Add the text into the Text of Question
                        if re.search(TagQEnd, block_item.text, flags=0) != None : # if there is a TagEnd of a multi-paragraph question in it
                            Multi_Paragraph = False # it is the end of Multi Pragraph
                            Go_DictionUID = True # Go to send the question to the dictionary and the AAP


                    #=======  1.2.4 - TREATMENT IF IT IS OK TO SEND QUESTION TO DICTIONARY AND AAP ========
                    if Go_DictionUID == True: # If OK to send the question to Dictionary and AAP

                        #=======  1.2.4.1 - RETREIVE THE SIZE OF ANSWER FROM THE PARAGRAPH (IF ANY) ========

                        #=======  CHECK IF THE TEXT OF THE QUESTION CONTAINS A REQUIREMENT FOR SIZE OF ANSWER AND IF YES, RETRIEVE IT =======
                        if OneOfTheWords_Is_InTheParagraph (Text_Question, list_of_SizeWords_OK, list_of_SizeWords_KO):
                        #======= Manage case of parenthesis in the text = probably a size requirement inside the parenthesis
                            if re.search(r'\(', Text_Question, flags=0)!= None : 
                            # A parenthesis in the text of the paragraph = good probability that it is for size of answer requirement
                                #======= extract the size information if it is inside the parenthesis
                                PosiStart = re.search(r'\(', Text_Question, flags=0).start() # start poition of '('
                                PosiEnd = re.search(r'\)', Text_Question, flags=0).start()+1 # end position of ')'
                                TheText = Text_Question[PosiStart+1:PosiEnd-1] # extract the size information which is betewwen the parenthesis and erase the parenthesis
                                if OneOfTheWords_Is_InTheParagraph (TheText, list_of_SizeWords_OK, list_of_SizeWords_KO):
                                # if there is size info in the paragraph, we must split it into Question + size info
                                    ListDict.clear() # Empty the list to put only a part of the paragraph in the question field instead of all the paragraph
                                    ListDict.append(Text_Question[:PosiStart]+' '+Text_Question[PosiEnd:]) # The 1st part + 3rd part of the paragraph is the question
                                    ListDict.append(TheText) # and then add the size information into the list 
                                else:# Finally, we thought it was a size requirement but we were wrong and it was not
                                    ListDict.append(Text_Question) # Put the question at first position of the list for dictionary
                                    ListDict.append('') # 2nd position of the List "size of answer" is empty
                            else:# No "(" so size requirement is empty)
                                ListDict.append(Text_Question) # Put the question at first position of the list for dictionary
                                ListDict.append('') # 2nd position "size of answer" is empty

                        else: # if no size of answer, put empty size
                            ListDict.append(Text_Question) # Put the question at first position of the list for dictionary
                            ListDict.append('') # 2nd position "size of answer" is empty
                                                        
                        #======= 1.2.4.2 FILL THE EMPTY FIELDS OF THE LIST ==========
                        ListDict.append('') # position "general context"
                        ListDict.append('') # position "question context"
                        ListDict.append('') # position "Qualification Close-Open question"
                        ListDict.append('') # position "Anwser of the question"

                        #======= 1.2.4.3 INSERT THE UID AT THE RIGHT PLACE (below the question) =====
                        QuestionUI = uuid.uuid4().hex
                        Insert_Text_Paragraph (block_item, '' , '\n' + QuestionUI)
                        # We use the function Insert_Text_Paragraph to insert '\n' + QuestionUI at the end without loosing the look & feel of the paragraph

                        #======= 1.2.4.4 ADD THE UID + QUESTION & INFO INTO THE DICTIONARY ======
                        new_list = ListDict.copy() 
                        DictQuestions[QuestionUI] = new_list #add the list to the dictionary with a Unique ID
                        ListDict.clear() # Empty the list for next question
                        Go_DictionUID = False 
                        Text_Question = '' # Clear the variable Texte_Question



            #============== 2 - RETREIVE QUESTIONS & SIZE IN A CELL OF A TABLE ================================================    

                #=======  2.1 - RETREIVE THE QUESTIONS FROM THE TABLE CELLS  ========
                elif isinstance(block_item, Table): # treatment of a table with cells
                    for row in range(len(block_item.rows)): # Loop on all cells = all rows and all columns
                        for col in range(len(block_item.columns)): #questions are generally in the 1st column but we check "?" everywhere in the table
                            if block_item.cell(row, col).text.strip() != '': # if the cell is not empty

                                #  If "?" in the cell, it is a question to be tagged
                                if re.search(r'\?', block_item.cell(row, col).text, flags=0)!= None : # if there is a "?" in it
                                    ListDict.clear() # Empty the list for next question
                                    ListDict.append(block_item.cell(row, col).text) # add the question to the list


                                    #=======  2.2 - RETREIVE THE SIZE OF ANSWER (IF ANY) FROM THE TABLE CELLS IF A QUESTION HAS BEEN FOUND ========

                                    #=======  CHECK IF THE CELL CONTAINS A REQUIREMENT FOR SIZE OF ANSWER AND IF YES, RETRIEVE IT =======
                                    if OneOfTheWords_Is_InTheParagraph (block_item.cell(row, col).text, list_of_SizeWords_OK, list_of_SizeWords_KO):
                                    #======= Manage case of parenthesis in the text = probably a size requirement inside the parenthesis
                                        if re.search(r'\(', block_item.cell(row, col).text, flags=0)!= None : 
                                        # A parenthesis in the text of the paragraph = good probability that it is for size of answer requirement
                                            #======= extract the size information if it is inside the parenthesis
                                            PosiStart = re.search(r'\(', block_item.cell(row, col).text, flags=0).start() # start poition of '('
                                            PosiEnd = re.search(r'\)', block_item.cell(row, col).text, flags=0).start()+1 # end position of ')'
                                            TheText = block_item.cell(row, col).text[PosiStart+1:PosiEnd-1] # extract the size information which is betewwen the parenthesis and erase the parenthesis
                                            if OneOfTheWords_Is_InTheParagraph (TheText, list_of_SizeWords_OK, list_of_SizeWords_KO):
                                            # if there is size info in the paragraphe, we must split it into Question + size info
                                                ListDict.clear() # Empty the list to put only a part of the paragraph in the question field instead of all the paragraph
                                                ListDict.append(block_item.cell(row, col).text[:PosiStart]) # The 1st part of the paragraph is the question
                                                ListDict.append(TheText) # and then add the size information into the list 
                                            else:# Finally, we thought it was a size requirement but we were wrong and it was not
                                                ListDict.append('') # position "size of answer" is empty
                                        else:# No "("" so size requirement is empty
                                            ListDict.append('') # position "size of answer" is empty
                                    else: # if no size of answer, put empty size
                                        ListDict.append('') # position "size of answer" is empty

    #!!!!!!!!!!!!!!! TO DO Here = else, if no size in the paragraph, we should check the next paragraph to see if it contains Size info



                                    #======= 2.3 FILL THE EMPTY FIELDS OF THE LIST ==========
                                    ListDict.append('') # position "general context"
                                    ListDict.append('') # position "question context"
                                    ListDict.append('') # position "Qualification Close-Open question"
                                    ListDict.append('') # position "Anwser of the question"

                                    #======= 2.4 INSERT THE UID AT THE RIGHT PLACE (below the question) =====
                                    QuestionUI = uuid.uuid4().hex
                                    #======= Case of Table with only 1 column =====
                                    if len (block_item.columns) == 1 : # If there is only one column 
                                        if len (block_item.rows) > row and block_item.cell(row+1, col).text.strip() == '': #there is a column below and it is empty
                                            block_item.cell(row+1, col).text = block_item.cell(row+1, col).text + QuestionUI 
                                            # put the UID in the empty cell below the current one (no insert so the format can be modified)
                                        else: # else, put the UID at the end of the current cell
                                            Insert_Text_Cell (block_item.cell(row, col), '' , '\n' + QuestionUI)
                                    #======= Case of Table with more than 1 column =====
                                    elif len (block_item.columns) > 1 : # If there is more than one column 
                                        if len (block_item.columns) > col+1 :
                                            if block_item.cell(row, col+1).text.strip() == '': #there is a column at the right of the current column and it is empty
                                                block_item.cell(row, col+1).text = block_item.cell(row, col+1).text + QuestionUI 
                                                # put the UID in the column at the right of the current one (no insert so the format could be modified)
                                            else: # else, put the UID at the end of the current cell
                                                Insert_Text_Cell (block_item.cell(row, col), '' , '\n' + QuestionUI)
                                                # We use the function Insert_Text_Cell to insert '\n' + QuestionUI at the end without loosing the look & feel of the paragraph
                                        else: # else, put the UID at the end of the current cell
                                            Insert_Text_Cell (block_item.cell(row, col), '' , '\n' + QuestionUI)
                                    # We use the function Insert_Text_Cell to insert '\n' + QuestionUI at the end without loosing the look & feel of the paragraph

                                    #======= 2.5 ADD THE UID + QUESTION & INFO INTO THE DICTIONARY ======
                                    new_list = ListDict.copy() 
                                    DictQuestions[QuestionUI] = new_list #add the list to the dictionary with a Unique ID
                                    ListDict.clear() # Empty the list for next question

            print(DictQuestions)
            docWork.save(PathFolderSource + r'/' + NameOfWorkDocument + '-with UID.docx')
        else:
            MessageError = str(datetime.now()) + ' Error encountered when reading Word docx file , please check type .docx and name of the file with no UID)' 
            logging.error(MessageError)
            print(MessageError)

    
    print('End of the read program')



    return DictQuestions
#@@@@@@@@@@@@@@@@@@@@@@@ END OF "READ QUESTIONS AND SIZE ANSWER REQUIREMENTS IN NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "WRITE ANSWERS INTO NEW AAP AND Q&A DOCX FILE" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
def Write_Answers_in_docx (Dict_UIDQuestionsSizeAnwsers, PathFolderSource,  PathForOutputsAndLogs, TagQStart = "<>", TagQEnd = "</>" ):
    """
    CONTEXT:
    Writes answers below or near each question in a docx file AAP ("Appel A Projet")
    Questions and answers associated with Unique IDs (UID) are received in a dictionary 
    An AAP file with de questions associated to UID has already been created by the read function 
        and will be used by the current Write function
    ACTIONS OF THE CODE
    Finds UID inside the AAP document and replace it by the answer associated with the UID from the dictionary

    Args:
        Dict_UIDQuestionsSizeAnwsers: The dictionary containing the UID + questions + Answers
        PathFolderSource: Path to the folder containing the files to be read
        PathForOutputsAndLogs: Path to the folder containing the log file
        TagQStart = "<>" Tag indicating the beginning of a Multi-paragraphs question (question with context below)
        TagQEnd = "</>" Tag indicating the end of a Multi-paragraphs question (question with context below) 

    Returns:
        The function returns nothing but creates 2 files 
        1 file is the AAP with the answers inside associated with the corresponding questions
        1 file is a simple docx file containing questions and answers
        It also logs errors in a file named "logs-IA_for_Asso.txt" in the folder "PathForOutputsAndLogs"
    """

    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)

    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary            
                    
                    if re.search(r'UID', NameOfDocument, flags=0)!= None : # if there is a "UID" in the name of the file
                    # Here, we want to open the file AAP with UID, so we only work with a file including "UID" in its name

                        # for each key of the dictionary, corresponding to the document
                        # find the key in the document and replace it by the answer near the question

                        #============== 1 - SUPPRESS "??", tags "<>" and "</>" IN A "FULL TEXT" PARAGRAPH (NOT IN A TABLE)================================================    
                        for docpara in document.paragraphs:
                            if "??" in docpara.text: # Suppress "??" added to identify questions
                                Delete_Text_Paragraph (docpara, "??")
                                # Do it before inserting the answer to avoid modifying the answer
                            if TagQStart in docpara.text: # Suppress TagQStart added to identify questions
                                Delete_Text_Paragraph (docpara, TagQStart)
                                # Do it before inserting the answer to avoid modifying the answer
                            if TagQEnd in docpara.text: # Suppress TagQEnd added to identify questions
                                Delete_Text_Paragraph (docpara, TagQEnd)
                                # Do it before inserting the answer to avoid modifying the answer

                        #============== 2 - REPLACE UID BY THE ANSWER IN A "FULL TEXT" PARAGRAPH (NOT IN A TABLE)================================================    
                        # Now, we replace the UID keys by the answers in the full text of the document
                        for docpara in document.paragraphs:
                            for key, value in Dict_UIDQuestionsSizeAnwsers.items() :
                                if key in docpara.text: # key is the UID
                                    Insert_Text_Paragraph (docpara, "" , '\n' + value[5] )# Insert the answer
                                    # Suppress key because we have already inserted the answer
                                    Delete_Text_Paragraph (docpara, key)

                        #============== 3 - SUPPRESS "??", tags "<>" and "</>" IN A CELL OF A TABLE)================================================    
                        for index, table in enumerate(document.tables):
                            for row in range(len(table.rows)):
                                for col in range(len(table.columns)):
                                    if "??" in table.cell(row, col).text: # Suppress "??" added to identify questions
                                        #table.cell(row, col).text = table.cell(row, col).text.replace("??", "") # suppress the UID
                                        Delete_Text_Cell (table.cell(row, col), "??")
                                    if TagQStart in table.cell(row, col).text: # Suppress TagQStart added to identify questions
                                        #table.cell(row, col).text = table.cell(row, col).text.replace(TagQStart, "") # suppress the UID
                                        Delete_Text_Cell (table.cell(row, col), TagQStart)
                                    if TagQEnd in table.cell(row, col).text: # Suppress TagQEnd added to identify questions
                                        #table.cell(row, col).text = table.cell(row, col).text.replace(TagQEnd, "") # suppress the UID
                                        Delete_Text_Cell (table.cell(row, col), TagQEnd)

                        #============== 4 - REPLACE UID BY THE ANSWER IN A CELL OF A TABLE  ================================================    
                        # then, we replace the keys by the answers in the tables of the document
                        for index, table in enumerate(document.tables):
                            for key, value in Dict_UIDQuestionsSizeAnwsers.items():
                                for row in range(len(table.rows)):
                                    for col in range(len(table.columns)):
                                        if key in table.cell(row, col).text:
                                            #table.cell(row, col).text = table.cell(row, col).text.replace(key, value)
                                            Insert_Text_Cell (table.cell(row, col),  "" ,  value[5] )# Insert the answer
                                            Delete_Text_Cell (table.cell(row, col), key)
                                            #table.cell(row, col).text = table.cell(row, col).text.replace(key, "")# suppress the UID


                        print("==========    DICTIONNAIRE AVEC ANSWERS :   ======")
                        print(Dict_UIDQuestionsSizeAnwsers)

                        #============== 3 - CREATE AAP WITH ANSWERS FILE ================================================    
                        # We create a new version of the AAP document with the answers
                        document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument[:-13] + "_with_answers" + '_' + str(datetime.now())[:-7] + '.docx' )

                        #============== 4 - CREATE A SIMPLE DOCX FILE WITH QUESTIONS AND ANSWERS ================================================    
                        # We create a new document containing only the questions and answers
                        documentQA = Document()
                        documentQA.add_heading('List of questions and answers of ' + NameOfDocument[:-14] + ' ' + str(datetime.now())[:-7], 0)

                        for key, value in Dict_UIDQuestionsSizeAnwsers.items():
                            p = documentQA.add_paragraph()
                            if "??" in value[0]: # Suppress from Value[0] "??" added to identify questions
                                value[0] = value[0].replace("??", "") # suppress "??"
                            if TagQStart in value[0]: # Suppress TagQStart added to identify questions
                                value[0] = value[0].replace(TagQStart, "") # suppress TagQStart
                            if TagQEnd in value[0]: # Suppress TagQEnd added to identify questions
                                value[0] = value[0].replace(TagQEnd, "") # suppress TagQEnd
                            Therun = p.add_run(value[0])
                            Therun.bold = True
                            Therun.font.color.rgb = RGBColor(255, 0, 0)
                            documentQA.add_paragraph('\n' + value[5] + '\n')
                            documentQA.save(Path_where_we_put_Outputs+ r'/' + NameOfDocument[:-14]+ '_Q-A' + '_' + str(datetime.now())[:-7] + '.docx' )
                
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when opening for writing the Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

    print('End of the write program')
    return
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "WRITE ANSWERS INTO NEW AAP AND Q&A DOCX FILE" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Settings for the path files
Path_where_we_put_Outputs = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/Pour les logs/' 
Folder_where_the_files_are = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/LesFilesA Lire/'

# imports
from docx import Document # import de python-docx
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
from docx.shared import RGBColor

import re
from datetime import datetime # for log file
import logging # for log file
import uuid # for unique ID creation
import glob # for file opening & reading

#activate logging of errors in a txt file
logging.basicConfig(filename=Path_where_we_put_Outputs + r'/logs-IA_for_Asso.txt')

# initialize variables
list_of_SizeWords_OK = [
     " MAX", " MIN", " CARACT", " CHARACT", " LIGNE", " LINE", " SIGN", " PAGE",  " PAS EXC", " NOT EXCEED"
         ]

list_of_SizeWords_KO = [
     " SIGNAT", " MAXIMI", " MONTH", " MOIS", " ANS", " ANNé", " YEAR",  " DAY", " JOUR",
     " DURéE", " DURATION", " IMPACT", " AMOUNT", " MONTANT"
         ]
TagQStart = "<>"
TagQEnd = "</>"


# Read the questions in the files and create a dictionary with questions for IA (questions = where there is a question mark "?")
Dict_UIDQuestionsSize = Read_Questions_in_docx ( Folder_where_the_files_are, Path_where_we_put_Outputs, list_of_SizeWords_OK, list_of_SizeWords_KO, TagQStart , TagQEnd )
# TODO : Send DictQuestionsSizeAnswers to Streamlit

# For the moment, we create a dictionary of answers with the same keys as the dictionary of questions
# by just taking the question as the answer we just put "ANSWER TO: " + the question
Dict_UIDQuestionsSizeAnswers = Dict_UIDQuestionsSize.copy()
for key, value in Dict_UIDQuestionsSizeAnswers.items():
    value[5]="ANSWER TO "+value[0]
     
# Write the answers into the docx files just below the questions
Write_Answers_in_docx (Dict_UIDQuestionsSizeAnswers, Folder_where_the_files_are, Path_where_we_put_Outputs, TagQStart , TagQEnd )
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


Le but de ce notebook est de faire une première pipeline qui, à partir d'un ensemble typique de documents, génère la demande de financements souhaitée.

Before reading an AAP document for IA, it must be pre-tagged to identify questions in it. The Pretag function below allows to pretag the document automatically to facilitate the pretag of AAP. When this pretag function has finished, the user can read the result in Microsoft Word and correct/complete the pretag of the questions. This function pretags in the full text ans also in the tables of the document. The criteria to identify questions are either a "?" in the text or key words idicting a question such as : "explain", "describe",..

In [ ]:
#PRETAG QUESTIONS IN NEW AAPs
# ========================================================================================================================================================
# this program has a function that pre-tags questions in  .docx files contained in a folder
# The questions are identified by the question mark "?" and a tag is added at the beginning and at the end of the question in the docx files.
# this function also reads tables in the .docx files to identify the questions there 
#
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ TESTS & IMPROVEMENTS NEEDED @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#Improvement to bring: the "zone with no question" works for full text but not for tables. Need to make it work for tables
#Improvement to bring: the key words list indicating a question should be adapted to each donator because each one has its own vocbulary
# so we should add the possibility for the user to choose the question key words he wants to use in the pretag activity
# #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END TESTS & IMPROVEMENTS NEEDED @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
import re
from docx.shared import RGBColor
from docx.shared import Pt
from docx.dml.color import ColorFormat
from docx.enum.style import WD_STYLE_TYPE
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "PRETAG ONE QUESTION FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Function to pretag a question : text of the question given in input, return the pretagued text of the question
def Pretag_One_Question (QuestionToPretag, TagStartQuestion, TagEndQuestion):
# This function pretags a question by adding a tag at the beginning and at the end of the question
    return TagStartQuestion + QuestionToPretag + TagEndQuestion
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "PRETAG ONE QUESTION FUNCTION @


#@@@@@@@@@@@@@@@@@@@@@@@@@@@ "PRETAG QUESTIONS IN NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Function to read the questions inside files with .docx extension contained in a folder using tags
def Pretag_Questions_in_docx (PathFolderSource, PathForOutputsAndLogs, list_of_QuestionWords, TagStartProjectQuestion, TagEndProjectQuestion, TagStart_NotForQuestions, TagEnd_NotForQuestions, GoTable = True ):
    """
        Uses python-docx 1.1.2 to manipulate Word documents : .docx only but not .doc. You need first to type "pip install python-docx" in your terminal

        Reads the content of files with .docx extension contained in a folder 
        Tags the questions found in the text of the document by adding a tag TagStartProjectQuestion at the beginning of the question and a tag TagEndProjectQuestion at the end of the question
        It does not tag questions in certain zones of the document identified to be without questions 
        Those zones without questions are identified by the tags TagStart_NotForQuestions at their beginning 
        and TagEnd_NotForQuestions at the end. The zones without questions correspond to parts of the AAP document
        which are guidance given by the donator and which do not contain questions

        How does it idenfiy questions ?
        1°) by looking for a question Mark "?" in the text of each paragraph
        2°) also when a Question Keyword is inside the text the paragraph (e.g. "describe", "explain",...)
        3°) In the tables of the document, it also identifies the questions in the cells of the standard tables, without looking for "?" or Keyword
        but by the shape of the table : it manages 3 standard types of tables and other types of tables are ignored
            *****************type 0 : table with only one column
                the first row is the question and the row below is waiting for the answer
                the row below must be empty (if it contains additional information, the function will not manage it properly)
                It writes the tagged "question" into the empty row at the place where the answer will be written afterwards

            *****************type 1 : table with two columns
                the first column is the question and the second column is for the answer
                the second column is generally empty but can sometimes contain additional information
                It retrieves then the content of 1srt column concatenated with de content of 2nd column and it writes the tagged question in the 2nd column
                and it writes the tagged "question" into the the 2nd column at the place where the answer will be written afterwards
                WARNING : if a table with 2 columns not empty is not for questions, this function will loose the content of the 2nd column, which is wrong
                In that case, the table must be tagged manually by TagStart_NotForQuestions in the cell Row=0, Column = 0 before launching the function, so as to avoid this issue
                This tag will then be removed by the function writing the answers in the final document

            TYPE 2 NOT MANAGED AT PRETAG LEVEL - WILL BE MANAGED WHEN READING AND CREATING DICTIONNARY OF QUESTIONS
            ******************type 2 : table with more than two columns and the first row not empty and the first column not empty
                this is a standard matrix table with information in rows and columns, 
                and answers awaited at the crossing of rows and columns
                the "question" retrieved is then the content of row 0 column 0 (title of the table)
                concatenated with the content of row 0 column X  (X going from 1 to the max column number)
                concatenated with the content of row Y column 0 (Y going from 1 to the max row number)
                and the corresponding tagged question shall be put in row Y column X
            ******************type 2 variant :
                generally, only the first row is not empty but 
                sometimes, the second row is also not empty = when there are merged cells in the first row 
                and the second row is a sub decomposition of the first row (e.g. 1srt row Year and 2nd row Month)
                the "question" retrieved is then the content of row 0 column 0 (title of the table)
                concatenated with the content of row 0 column X  (X going from 1 to the max column number)
                concatenated with the content of row 1 column X  (X going from 1 to the max column number)
                concatenated with the content of row Y column 0 (Y going from 1 to the max row number)
                and the corresponding tagged question shall be put in row Y column X


        Args:
            PathFolderSource: Path to the folder containing the files to be read
            PathForOutputsAndLogs: Path to the folder containing the log file
            list_of_QuestionWords: list of Question Keyword to be found inside the text (e.g. "describe", "explain",...)
            TagStartProjectQuestion: tag to be added at the beginning of the question
            TagEndProjectQuestion: tag to be added at the end of the question
            GoTable: Tag that indicates if we want to tag the questions in the tables of the document (default = True => by default, we tag also the questions in tables)
            TagStart_NotForQuestions: Tag that indicates the start of a zone of the document that does not contain questions
            #(e.g. part of the document reserved for the donator or which gives guidance for the answers)
            TagEnd_NotForQuestions: Tag that indicates the end of a section which has no question (e.g. : giving guidance to the NGO)

            
        Returns:
            The function returns nothing but creates a new version of each document that has been read, where each question is tagged at its beginning and end,
            Name of the new document = NameOfDocument+'-avecTags'
            It also logs errors in a file named "logs-IA_for_Asso.txt" in the folder "PathForOutputsAndLogs"
        """

    #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)

    # initialize variables
    TheTextofTheQuestion = '' # Text of a question
    NbQuestionsText = 0 # Number of questions identified in the full text (excluding tables)
    NbQuestionTables = 0 # Number of questions identified in the tables (excluding full text)
    We_Are_In_a_Zone_WithQuestion = True # Tag that indicates if the function is working in a zone of the document that contains questions or not
    #(e.g. part of the document reserved for donator or that gives guidance for the answers)

    # read content of the files, only if they are .docx (extension to other file types possible with the match - case)
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary
                    for Mystyle in document.styles:
                        print(Mystyle.name)

                       
                    # Here we manahge the questions in the full text of the document
                    for docpara in document.paragraphs:

                        # Here, we identify if we are in a zone of the document that contains questions or not
                        if re.search(TagStart_NotForQuestions, docpara.text, flags=0)!= None: # if we enter a zone without questions
                            We_Are_In_a_Zone_WithQuestion = False
                        if re.search(TagEnd_NotForQuestions, docpara.text, flags=0)!= None: # if we exit a zone without questions
                            We_Are_In_a_Zone_WithQuestion = True

                        # Here, we tag the questions identified by "?" in the full text of the document
                        if re.search(r'\?', docpara.text, flags=0)!= None and We_Are_In_a_Zone_WithQuestion == True: # If the text contains a question mark and the section is OK for questions
                            TheTextofTheQuestion = docpara.text
                            # we tag only if also it has not already been tagged
                            if re.search(TagStartProjectQuestion, TheTextofTheQuestion, flags=0)== None:
                                #QuestionTagged = Pretag_One_Question (TheTextofTheQuestion, TagStartProjectQuestion, TagEndProjectQuestion) 
                                # to avoid losing farmating of the paragraph when replacing the text
                                
                                # the code below allows to insert the tags in the paragraph
                                # without loosing the initial look & feel of the texte (size, font, color,..)
                                # because any other way of changing the text will unfortunately loose all of that

                                # insert the start tag
                                docpara.runs[0].text = docpara.runs[0].text.replace("", TagStartProjectQuestion,1) 
                                # then insert the end tag
                                NbRuns = docpara.runs.__len__()
                                docpara.runs[NbRuns-1].text = docpara.runs[NbRuns-1].text.replace(docpara.runs[NbRuns-1].text, docpara.runs[NbRuns-1].text + TagEndProjectQuestion,1)

                                
                                NbQuestionsText += 1 # Number of questions incremented

                        # then, we tag questions containing Words that often indicate a question or prompt
                        for keyword in list_of_QuestionWords:
                            #if the keyword in lowercase is in the text in lowercase, it is probably a question
                            if re.search(keyword.lower(), docpara.text.lower(), flags=0)!= None and We_Are_In_a_Zone_WithQuestion == True: # If the cell contains a question mark and the section is OK for questions
                                TheTextofTheQuestion = docpara.text
                                # we tag only if also it has not already been tagged    
                                if re.search(TagStartProjectQuestion, TheTextofTheQuestion, flags=0)== None:
                                    #QuestionTagged = Pretag_One_Question (TheTextofTheQuestion, TagStartProjectQuestion, TagEndProjectQuestion) 
                                    #docpara.text = QuestionTagged

                                    # insert the start tag
                                    docpara.runs[0].text = docpara.runs[0].text.replace("", TagStartProjectQuestion,1) 
                                    # then insert the end tag
                                    NbRuns = docpara.runs.__len__()
                                    docpara.runs[NbRuns-1].text = docpara.runs[NbRuns-1].text.replace(docpara.runs[NbRuns-1].text, docpara.runs[NbRuns-1].text + TagEndProjectQuestion,1)

                                    NbQuestionsText += 1 # Number of questions incremented

                    # Here we manage the questions in the Tables of the document, if the tag GoTable is True
                    if GoTable == True:

                        # Here, we tag the questions identified by "?"in the tables of the document
                        for index, table in enumerate(document.tables):

                            # Here, if the cell row0 Column 0 of the table has a flag "not for questions", we change the tag to ignore the table
                            if re.search(TagStart_NotForQuestions, table.cell(0, 0).text, flags=0)!= None: 
                                We_Are_In_a_Zone_WithQuestion = False

                            if (We_Are_In_a_Zone_WithQuestion == True) : # If the table is OK for questions

                                # Here we tag the questions of the table that contain a "?" if not already tagged
                                for row in range(len(table.rows)):
                                    for col in range(len(table.columns)):
                                        if re.search(r'\?', table.cell(row, col).text, flags=0)!= None : #re.match(r'?', table.cell(row, col).text): # If the cell contains a question mark
                                            TheTextofTheQuestion = table.cell(row, col).text
                                            # we tag only if also it has not already been tagged
                                            if re.search(TagStartProjectQuestion, TheTextofTheQuestion, flags=0)== None:

                                                # the code below allows to insert the tags in the cell
                                                # without loosing the initial look & feel of the texte (size, font, color,..)
                                                # because any other way of changing the text will unfortunately loose all of that
                                                
                                                # scan the paragraphs of the cell and insert the tags
                                                ListOfRuns = []
                                                for paragCell in table.cell(row, col).paragraphs:
                                                    ListOfRuns.extend(paragCell.runs)
                                                NbRuns = len(ListOfRuns)
                                                ListOfRuns[0].text = ListOfRuns[0].text.replace("", TagStartProjectQuestion,1) 
                                                ListOfRuns[NbRuns-1].text = ListOfRuns[NbRuns-1].text.replace(ListOfRuns[NbRuns-1].text, ListOfRuns[NbRuns-1].text + TagEndProjectQuestion,1)

                                                NbQuestionTables += 1 # Number of questions incremented

                                # then here, we tag the questions by their position in standard tables without checking "?" in the text of the cell
                                # Except if they have already been tagged because of "?" in the text of the cell  
                                NBColumns = len(table.columns)
                                if NBColumns == 1: # it is a "type 0" table 
                                    print("Type 0 table")
                                    col = 0
                                    for row in range(len(table.rows)):
                                        if table.cell(row, col).text.strip(" ") != '' and re.search(TagStartProjectQuestion, table.cell(row, col).text, flags=0)== None:# if the cell is not empty, it is a "question"
                                            # we tag only if also it has not already been tagged
                                            TheTextofTheQuestion = table.cell(row, col).text
                                            if re.search(TagStartProjectQuestion, TheTextofTheQuestion, flags=0)== None:

                                                # scan the paragraphs of the cell and insert the tags
                                                ListOfRuns = []
                                                for paragCell in table.cell(row, col).paragraphs:
                                                    ListOfRuns.extend(paragCell.runs)
                                                NbRuns = len(ListOfRuns)
                                                ListOfRuns[0].text = ListOfRuns[0].text.replace("", TagStartProjectQuestion,1) 
                                                ListOfRuns[NbRuns-1].text = ListOfRuns[NbRuns-1].text.replace(ListOfRuns[NbRuns-1].text, ListOfRuns[NbRuns-1].text + TagEndProjectQuestion,1)
                                                
                                                NbQuestionTables += 1 # Number of questions incremented
                      
                                if NBColumns == 2: # it is a "type 1" table 
                                    print("Type 1 table")
                                    col = 0
                                    for row in range(len(table.rows)):
                                        if table.cell(row, col).text.strip(" ") != '' and re.search(TagStartProjectQuestion, table.cell(row, col).text, flags=0)== None:
                                            # we tag only if also it has not already been tagged
                                            TheTextofTheQuestion = table.cell(row , col).text #+ ' ' + table.cell(row , col + 1).text # concatenate the 2 columns
                                            # At this stage, we do not concatenate cells contents and just put tags
                                            # the concatenate of some cells will be done at the read stage when we create the dictionnary of questions
                                            if re.search(TagStartProjectQuestion, TheTextofTheQuestion, flags=0)== None:
                                                #QuestionTagged = Pretag_One_Question (TheTextofTheQuestion, TagStartProjectQuestion, TagEndProjectQuestion) 
                                                #table.cell(row, col).text = QuestionTagged # replace the text in the cell by the text+tags                       
                                                
                                                # At this stage, we do not concatenate de cells content, we will do it when reading to create the dictionnary
                                                # scan the paragraphs of the cell and insert the tags + the text from the second column
                                                # scan the paragraphs of the cell and insert the tags
                                                ListOfRuns = []
                                                for paragCell in table.cell(row, col).paragraphs:
                                                    ListOfRuns.extend(paragCell.runs)
                                                NbRuns = len(ListOfRuns)
                                                ListOfRuns[0].text = ListOfRuns[0].text.replace("", TagStartProjectQuestion,1) 
                                                ListOfRuns[NbRuns-1].text = ListOfRuns[NbRuns-1].text.replace(ListOfRuns[NbRuns-1].text, ListOfRuns[NbRuns-1].text + TagEndProjectQuestion,1)





                                                #for paragCell in table.cell(row, col).paragraphs:
                                                #    # insert the start tag
                                                #    paragCell.runs[0].text = paragCell.runs[0].text.replace("", TagStartProjectQuestion,1) 
                                                #    # then insert the end tag
                                                #    NbRuns = paragCell.runs.__len__()
                                                    #paragCell.runs[NbRuns-1].text = paragCell.runs[NbRuns-1].text.replace(paragCell.runs[NbRuns-1].text, paragCell.runs[NbRuns-1].text + TagEndProjectQuestion,1)
                                                
                                                NbQuestionTables += 1 # Number of questions incremented
                                
                                # At this stage, we don not pre-tag the type 2 tables
                                # Il will be treated when we read the final tagged document to create the dictionnary of questions
                                # For more than 2 columns, we consider only the case of a "type 2" table (matrix table)
                                #  when row 0 is not empty and col 0 is not empty and we ignore the other cases
                                # we test uniquely the first row 2nd col (row = 0 col = 1) and the first column 2nd row (row 1 & col=0)
                                #if (NBColumns >2) and (table.cell(0, 1).text.strip(" ") != '') and (table.cell(1, 0).text.strip(" ") != ''):
                                #   print("Type 2 table")
                                    # A FAIRE : Gérer les cas où la 2ème ligne n'est pas vide et est une sous décomposition de la 1ère ligne
                                    # ---------------- CASE TYPE 2 STANDARD WITH ONLY 1 ROW OF TITLES----------------
                                #    if (table.cell(1, 1).text.strip(" ") == ''): # if the second Row is empty = it is a standard matrix table Type2
                                #        for row in range(1, len(table.rows) ):   # From second row (1) to max row 
                                #            for col in range(1, len(table.columns) ):  #  From second col (1) to max col
                                #                if table.cell(row, col).text.strip(" ") == '':
                                #                    TheTextofTheQuestion = table.cell(0,0).text + " " + table.cell(0,col).text + " " + table.cell(row, 0).text
                                                    # we tag only if also it has not already been tagged
                                #                    if re.search(TagStartProjectQuestion, TheTextofTheQuestion, flags=0)== None:
                                #                        QuestionTagged = Pretag_One_Question (TheTextofTheQuestion, TagStartProjectQuestion, TagEndProjectQuestion) 
                                #                        table.cell(row, col).text = QuestionTagged # replace the text in the cell by the text+tags                       
                                #                        NbQuestionTables += 1 # Number of questions incremented
                                    # ---------------- CASE TYPE 2 VARIANT WITH 2 ROWS OF TITLES ----------------
                                    # the second row is also not empty = when there are merged cells in the first row 
                                    # and the second row is a sub decomposition of the first row (e.g. 1srt row Year and 2nd row Month)
                                #    if (table.cell(1, 1).text.strip(" ") != ''): # if the second Row is not empty = it is a variant matrix table Type2
                                #        for row in range(2, len(table.rows) ):   # From third row (2) to max row 
                                #            for col in range(1, len(table.columns) ):  #  From second col (1) to max col
                                #                if table.cell(row, col).text.strip(" ") == '':
                                                    # we tag only if also it has not already been tagged
                                #                    TheTextofTheQuestion = table.cell(0,0).text + " " + table.cell(0,col).text + " " + table.cell(1,col).text + " " + table.cell(row, 0).text
                                                    # we tag only if also it has not already been tagged
                                #                    if re.search(TagStartProjectQuestion, TheTextofTheQuestion, flags=0)== None:
                                #                        QuestionTagged = Pretag_One_Question (TheTextofTheQuestion, TagStartProjectQuestion, TagEndProjectQuestion) 
                                #                        table.cell(row, col).text = QuestionTagged # replace the text in the cell by the text+tags                       
                                #                        NbQuestionTables += 1 # Number of questions incremented


                    print("Nbr of text questions = "+str(NbQuestionsText)+" and Nbr of table questions =  "+str(NbQuestionTables) + " in document " + NameOfDocument, end='\n')
                    print()
                    
                    document.save(PathFolderSource + r'/' + NameOfDocument[:-5] + '-avecTags.docx')
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when reading Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

            case '.doc':
                print('Fichier DOC')# OPEN QUESTION: do we consider reading .doc files ?
            case _:
                print('Fichier non pris en charge')
                #OPEN QUESTION: do we consider reading other types of files below ?
                #'rtf', 'pdf', 'xls', 'xlsx', 'csv', 'ppt', 'pptx',
                #'odc','odf', 'odg', 'odm', 'odp', 'ods','odt', 'odx'
                # WE SHOULD CHECK ALL EXTENSIONS OF THE FILES CONTAINED IN THE FOLDER 
                # AND PROMPT A MESSAGE IF EXTENSION NOT MANAGED
    print('End of the pretag program')
    return 
#@@@@@@@@@@@@@@@@@@@@@@@ END OF "PRETAG QUESTIONS IN NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@




#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Settings for the path files
Path_where_we_put_Outputs = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/Pour les logs/' 
Folder_where_the_files_are = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/LesFilesA Lire/'
from docx import Document # import de python-docx

#TagStartGeneralQuestion = '<generalQuestion>' # Tag that indicates the start of a general question (information about the NGO,..)
#TagEndGeneralQuestion = '</generalQuestion>' # Tag that indicates the end of a general question
TagStartProjectQuestion = '<projectQuestion>' # Tag that indicates the start of a project question (information about a project proposed by the NGO)
TagEndProjectQuestion = '</projectQuestion>' # Tag that indicates the end of a project question
TagStart_NotForQuestions = '<notForQuestions>' # Tag that indicates the start of a zone of the document that does not contain questions
TagEnd_NotForQuestions = '</notForQuestions>' # Tag that indicates the end of a zone of the document that does not contain questions

list_of_QuestionWords = [
     "expliqu", "list", "défin", "compar", "analys", "indiqu",
     "evalu", "discut", "identif", "soulig", "résum", "présent",
     "état", "justif", "élabor", "spécif", "détail",
     "attend", "plan", "planning", "contact", "plait",
     "demandeu", "candidat", "estim", "quel", "qui", "quoi", "comment", "où", "quand",
     "activit", "organis", "situation", "adress",
     "impact", "budget", "durée", "financ",
     "veuillez", "décri", "fourni", "plan d’action ", "calendrier", "contact", "action",  "description", "thématique",
     "candidat", "introduire", "antécédent", "activité", "organisation", "situation",
     "impact", "budget", "durée", "soutien", "partena",
     "Nom du projet", "Objet du financement", "Représentant légal", "Adresse du siège social",
     "Nombre d'adhérents", "Nombre de bénévoles", "Nombre de salariés", "Nombre de personnes bénéficiaires",
     "structure", "écosystème", "Implantation géographique", "Public cible", "Statut juridique",
     "court descriptif", "informations générales"
         ]
                            # to do after, manage the case "english AAP"
                             #"describe", "explain", "list", "define", "compare", "analyze",
                             #"evaluate", "discuss", "identify", "outline", "summarize",
                             #"provide", "state", "justify", "elaborate", "specify",
                             #"expected", "plan", "schedule", "contact", "please",
                             #"applicant", "candidate", "introducer", "background",
                             #"activities", "organization", "situation", "address",
                             #"summary", "impact", "budget", "duration", "financing",
#A FAIRE DANS STREAMLIT: FOURNIR LA VALEUR TRUE OU FALSE DU TAG GOTABLE - E ATTENDANT, ON MET TRUE
GoTable = True # Tag that indicates if we want to tag the questions in the tables of the document

# Pretag the questions in the files and create a copy of the files with the questions pretagged
Pretag_Questions_in_docx (Folder_where_the_files_are, Path_where_we_put_Outputs, list_of_QuestionWords, TagStartProjectQuestion, TagEndProjectQuestion, TagStart_NotForQuestions, TagEnd_NotForQuestions, GoTable)


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



## Load documents

In [9]:
#LOAD-DOCUMENTS = READ QUESTIONS IN NEW AAPs AND WRITE ANSWERS PROPOSED BY AI INTO NEW AAPs
# ========================================================================================================================================================
# READ EMPTY AAPs : this program has a function that reads the questions in  .docx files contained in a folder
# and moves the questions into a dictionary with a unique ID (UID) for each question
# This UID is also writen below the question in the .docx files
# The questions are identified by tags at the beginning and at the end of the question in the docx files.
# this function also reads tables in the .docx files to retreive the questions contained in the tables (no tag necessary)
# only 3 types of standard tables are managed and the other types of tables are ignored
# the UID is written into the cells of the tables which are waiting for an answer
# =======================================================================================================================================================
# WRITE DOCUMENTS TO FILL ANSWERS IN EMPTY AAPs : This program has also a function that writes the answers to the questions into the .docx files
# using a dictionnary of answers associated to the same UID as the questions
# So, the answers are written into the .docx files, below the questions or inside the cells of the tables 
# 
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ TESTS & IMPROVEMENTS NEEDED @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# 1°) faire des tests avec vrais AAP nouveaux pour fiabiliser la lecture des tableaux
# 2°) Distinguer les type de doc lus: AAP pour AAP nouveau et AAPE + PP pour context IA et activer une pure lecture tabelau pour AAPE+PP
# 3°) Améliorer la détection des tables matricielles en contrôlant que toute la première ligne et toute la première colonne sont non vides
# 4°) Envoyer vraiment à l'IA les question des tableau matriciels pour vérifier la compréhension
# 5°) Mettre les bonnes valeurs des tags de questions defined by Kristin
# 6°) Gérer la distinction des tags généraux et des tags projets et envoyer vers 2 dictionnaires distincts ??
# 8°) Improve file error management : file not in the folder, not readable, not writable, not closed, not found
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END TESTS & IMPROVEMENTS NEEDED @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


#@@@@@@@@@@@@@@@@@@@@@@@@@@@ "READ QUESTIONS FROM NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Function to read the questions inside files with .docx extension contained in a folder using tags
def Read_Questions_From_docx (PathFolderSource, PathForOutputsAndLogs):
# This program reads the content of files with .docx extension contained in a folder
# It uses python-docx 1.1.2 to manipulate Word documents : .docx only but not .doc so you need first to type "pip install python-docx" in your terminal
# It identifies the questions from the other information by looking for the tag TagStartGeneralQuestion at the beginning of the question
# and for the tag TagEndGeneralQuestion at the end of the question (a question can have several paragraphs)
# TagStartGeneralQuestion indicates the Start of the Question and TagEndGeneralQuestion indicates the End of the Question
# The ouptput of this function is double :
# 1°) return a dictionary containing the questions for AI : Key= "NameOfFile - Unique ID" and Value = Text of the question
# 2°) create in a folder a new version of each document that has been read, where below each question,
#  is added the same Key "NameOfFile - Unique ID"
# After the answers are created, It will allow to insert the answers at the right place just below the corresponding question in the documents
# The user will then be able to see and modify in each document the original question and the answer given by the AI
# The function also logs errors in a file named "logs-IA_for_Asso.txt" in the folder "PathForOutputsAndLogs"

    # for unique ID creation
    import uuid

 
    #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)

    # initialize variables
    ItIsAQuestion = False # Tag that indicates if the current paragraph is inside a question
    TheTextofTheQuestion = '' # Text of a question
    DictQuestions = {} #initialise the dictionnary of questions
    TagStartGeneralQuestion = '<gquestion>' # Tag that indicates the start of a general question (information about the NGO,..)
    TagEndGeneralQuestion = '<gquestion/>' # Tag that indicates the end of a general question
    LenTagStartGeneralQuestion = len(TagStartGeneralQuestion) # length of the tag
    LenTagEndGeneralQuestion = len(TagEndGeneralQuestion) # length of the tag
    TagSartProjectQuestion = '<pquestion>' # Tag that indicates the start of a project question (information about a project proposed by the NGO)
    TagEndProjectQuestion = '<pquestion/>' # Tag that indicates the end of a project question

    # read content of the files, only if they are .docx (extension to other file types possible with the match - case)
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary

                    # here below, we retrieve the questions included in the tables of the document, 
                    # We manage 3 standard types of tables and other types of tables are ignored

                    # *****************type 0 : table with only one column
                    # the first row is the question and the row below is waiting for the answer
                    # the row below must be empty (if it contains additional information, the function will not manage it properly)
                    # the must be only 1 empty row below the question (if it is not the cas, the function will not manage it properly)
                    # the "question" retrieved is then the content of not empty row and the UID (for the answer) is written in the empty row

                    # *****************type 1 : table with two columns
                    # the first column is the question and the second column is for the answer
                    # the second column is generally empty but can sometimes contain additional information
                    # the "question" retrieved is then the content of 1srt column concatenated with de content of 2nd column

                    # ******************type 2 : table with more than two columns and the first row not empty and the first column not empty
                    # this is a standard matrix table with information in rows and columns, 
                    # and answers awaited at the crossing of rows and columns
                    # the "question" retrieved is then the content of row 0 column 0 (title of the table)
                    # concatenated with the content of row 0 column X  (X going from 1 to the max column number)
                    # concatenated with the content of row Y column 0 (Y going from 1 to the max row number)
                    # and the corresponding answer (UID) shall be put in row Y column X
                    # ******************type 2 variant :
                    # generally, only the first row is not empty but 
                    # sometimes, the second row is also not empty = when there are merged cells in the first row 
                    # and the second row is a sub decomposition of the first row (e.g. 1srt row Year and 2nd row Month)
                    # the "question" retrieved is then the content of row 0 column 0 (title of the table)
                    # concatenated with the content of row 0 column X  (X going from 1 to the max column number)
                    # concatenated with the content of row 1 column X  (X going from 1 to the max column number)
                    # concatenated with the content of row Y column 0 (Y going from 1 to the max row number)
                    # and the corresponding answer shall be put in row Y column X
                    for index, table in enumerate(document.tables):
                        NBColumns = len(table.columns)
                        if NBColumns == 1: # it is a "type 0" table 
                            print("Type 0 table")
                            for row in range(len(table.rows)):
                                if table.cell(row, 0).text.lstrip(" ") != '':# if the cell is not empty, it is a "question"
                                    ItIsAQuestion =True
                                    TheTextofTheQuestion = table.cell(row, 0).text 
                                    QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex # create a unique ID for the question
                                    DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                if ItIsAQuestion and row>=1 and table.cell(row, 0).text.lstrip(" ") == '' and table.cell(row-1, 0).text.lstrip(" ") != '':
                                    # if the cell is empty and the previous cell is not empty, the current cell is waiting for the answer of the question of the previous cell
                                    # so we write the UID of the previous question into the cell
                                    # ItIsAQuestion is tested to manage the case of several empty rows below a question
                                    table.cell(row, 0).text = QuestionUI
                                    ItIsAQuestion =False # to manage the case where the table has more than 1 empty row below a "question"
                                print("in row = "+str(row)+" and Col = "+str(0)+", the content is "+table.cell(row, 0).text, end='\n')
                       
                        if NBColumns == 2: # it is a "type 1" table 
                            print("Type 1 table")
                            for row in range(len(table.rows)):
                                if table.cell(row, 0).text.lstrip(" ") != '':
                                    TheTextofTheQuestion = table.cell(row , 0).text + ' ' + table.cell(row , 1).text # concatenate the 2 columns
                                    QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex # create a unique ID for the question
                                    DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                    table.cell(row, 1).text = QuestionUI # write the UID in the second column
                                print("in row = "+str(row)+" and Col = "+str(0)+", the content is "+table.cell(row, 0).text, end='\n')
                                print("in row = "+str(row)+" and Col = "+str(1)+", the content is "+table.cell(row, 1).text, end='\n')

                        # For more than 2 columns, we consider only the case of a "type 2" table (matrix table)
                        #  when row 0 is not empty and col 0 is not empty and we ignore the other cases
                        # we test uniquely the first row 2nd col (row = 0 col = 1) and the first column 2nd row (row 1 & col=0)
                        if (NBColumns >2) and (table.cell(0, 1).text.lstrip(" ") != '') and (table.cell(1, 0).text.lstrip(" ") != ''):
                            print("Type 2 table")
                            # A FAIRE : Gérer les cas où la 2ème ligne n'est pas vide et est une sous décomposition de la 1ère ligne
                            # ---------------- CASE TYPE 2 STANDARD WITH ONLY 1 ROW OF TITLES----------------
                            if (table.cell(1, 1).text.lstrip(" ") == ''): # if the second Row is empty = it is a standard matrix table Type2
                                for row in range(1, len(table.rows) ):   # From second row (1) to max row 
                                    for col in range(1, len(table.columns) ):  #  From second col (1) to max col
                                        if table.cell(row, col).text.lstrip(" ") == '':
                                            TheTextofTheQuestion = table.cell(0,0).text + " " + table.cell(0,col).text + " " + table.cell(row, 0).text
                                            QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex
                                            DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                            # question to the dictionary with a Unique ID
                                            table.cell(row, col).text = QuestionUI # put UID in the cell of the table
                            # ---------------- CASE TYPE 2 VARIANT WITH 2 ROWS OF TITLES ----------------
                            # the second row is also not empty = when there are merged cells in the first row 
                            # and the second row is a sub decomposition of the first row (e.g. 1srt row Year and 2nd row Month)
                            if (table.cell(1, 1).text.lstrip(" ") != ''): # if the second Row is not empty = it is a variant matrix table Type2
                                for row in range(2, len(table.rows) ):   # From third row (2) to max row 
                                    for col in range(1, len(table.columns) ):  #  From second col (1) to max col
                                        if table.cell(row, col).text.lstrip(" ") == '':
                                            TheTextofTheQuestion = table.cell(0,0).text + " " + table.cell(0,col).text + " " + table.cell(1,col).text + " " + table.cell(row, 0).text
                                            QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex
                                            DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                            # question to the dictionary with a Unique ID
                                            table.cell(row, col).text = QuestionUI # put UID in the cell of the table
                        print("Nbr of columns = "+str(len(table.columns))+" and Nbr of rows =  "+str(len(table.rows)), end='\n')
                        for row in range(len(table.rows)):
                            for col in range(len(table.columns)):
                                print("in row = "+str(row)+" and Col = "+str(col)+", the content is "+table.cell(row, col).text, end='\n')
                        print()
                    print()

                    # then here, we retrieve the questions identified by tags TagStartGeneralQuestion and TagEndGeneralQuestion in the full text of the document
                    for docpara in document.paragraphs:
                        if (docpara.text != ''): # we don't want to add empty paragraphs
                            if(docpara.text[:LenTagStartGeneralQuestion]==TagStartGeneralQuestion): # if first characters are TagStartGeneralQuestion, then it is the start of a question
                                ItIsAQuestion = True
                                TheTextofTheQuestion = docpara.text[LenTagStartGeneralQuestion:]# eliminate the n first characters which are the TAG TagStartGeneralQuestion
                            else:
                                if (ItIsAQuestion): # if we are inside a question
                                    TheTextofTheQuestion = TheTextofTheQuestion + ". "+ docpara.text
                            if (docpara.text[-LenTagEndGeneralQuestion:]==TagEndGeneralQuestion): # if the end of the paragraph is TagEndGeneralQuestion, then it is the end of the question
                                ItIsAQuestion = False
                                TheTextofTheQuestion = TheTextofTheQuestion[:-LenTagEndGeneralQuestion]# eliminate the n last characters which are the TAG TagEndGeneralQuestion
                                QuestionUI = NameOfDocument + ' - ' + uuid.uuid4().hex
                                DictQuestions[QuestionUI] = TheTextofTheQuestion #add the question to the dictionary with a Unique ID
                                docpara.text = docpara.text + '\n' + QuestionUI
                                #TO DO AFTER : manager les infos entre les questions si on doit les fournir à l'IA
                                #TO DO AFTER : dans un dictionaire de complément d'infos
                                #TO DO AFTER : Gérer les numérotations indentées qui sous-divisent les questions ?
                                #TO DO AFTER : Gérer les tableaux ?
                                #TO DO AFTER : Gérer la résistance à l'erreur = début TagStartGeneralQuestion mais manque fin TagEndGeneralQuestion ou inverse

                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument)
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when reading Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

            case '.doc':
                print('Fichier DOC')# OPEN QUESTION: do we consider reading .doc files ?
            case _:
                print('Fichier non pris en charge')
                #OPEN QUESTION: do we consider reading other types of files below ?
                #'rtf', 'pdf', 'xls', 'xlsx', 'csv', 'ppt', 'pptx',
                #'odc','odf', 'odg', 'odm', 'odp', 'ods','odt', 'odx'
                # WE SHOULD CHECK ALL EXTENSIONS OF THE FILES CONTAINED IN THE FOLDER 
                # AND PROMPT A MESSAGE IF EXTENSION NOT MANAGED
    print('End of the read program')
    return DictQuestions
#@@@@@@@@@@@@@@@@@@@@@@@ END OF "READ QUESTIONS FROM NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ "WRITE ANSWERS INTO NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Function to write the answer below each question inside files with .docx extension contained in a folder
def Write_Answers_in_docx (PathFolderSource, DictonaryOfAnswers, PathForOutputsAndLogs):
# The main program has already submitted each question to the AI 
# and filled the "DictonaryOfAnswers" with the answers to the questions 
# The "DictonaryOfAnswers" has the same Key "NameOfFile - Unique ID" as the "DictonaryOfQuestions"
# Then the main program will call the "Write_Answers_in_docx" function to write the answers 
# from the he "DictonaryOfAnswers" into the documents themselves
# As the read function has already placed the key of the question below the question, 
# this function will just have to find the key below the question and replace ti by the answer, back in the docx file 
# It will also remove the TagEndGeneralQuestion and TagEndGeneralQuestion tags from the questions


     #activate logging of errors in a txt file
    from datetime import datetime
    import logging
    logging.basicConfig(filename=PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt')

    # initialize variables
    TagStartGeneralQuestion = '<gquestion>' # Tag that indicates the start of a general question (information about the NGO,..)
    TagEndGeneralQuestion = '<gquestion/>' # Tag that indicates the end of a general question
    TagSartProjectQuestion = 'SQPR' # Tag that indicates the start of a project question (information about a project proposed by the NGO)
    TagEndProjectQuestion = 'EQPR' # Tag that indicates the end of a project question


    #Create a list of path to all the files (no hidden files) contained in the folder “PathFolderSource” 
    import glob
    FilesWithPath = []
    for file in glob.glob(PathFolderSource +'*.*'):
        FilesWithPath.append(file)
    #FilesWithPath.remove(PathForOutputsAndLogs + r'/logs-IA_for_Asso.txt') # remove the log file from the list of files to be read
    #TO DO AFTER : manage the case where the log file is not in the folder
    for file in FilesWithPath:
        TheExtension = file [-4:] 
        match TheExtension:
            case 'docx':
                try:
                    f = open(file, 'rb')
                    document = Document(f)
                    NameOfDocument = file.split('/')[-1] # Name of the file without the path will be used in the Key of the dictionnary

                    # for each key of the dictionary, corresponding to the document
                    # find the key in the document and replace it by the answer
                    # As the key was below the question, this puts the answer just below the question
                    # if the key is not found, log an error

                    # Create a subset of the dictionary corresponding to the document opened
                    Dict_Of_Answers_of_the_Document = dict(filter(lambda item: item[0].split(' - ')[0] == NameOfDocument, DictonaryOfAnswers.items()))
                    print(Dict_Of_Answers_of_the_Document) # The answer dictionnary for the document
                    
                    # Now, we replace the keys by the answers in the full text of the document
                    for docpara in document.paragraphs:
                        for key, value in Dict_Of_Answers_of_the_Document.items():
                            if key in docpara.text:
                                docpara.text = docpara.text.replace(key, value)
                                # Dict_Of_Answers_of_the_Document.pop(key) # remove the key from the dictionnary when it has been found

                    # then, we replace the keys by the answers in the tables of the document
                    for index, table in enumerate(document.tables):
                        for key, value in Dict_Of_Answers_of_the_Document.items():
                            for row in range(len(table.rows)):
                                for col in range(len(table.columns)):
                                   if key in table.cell(row, col).text:
                                       table.cell(row, col).text = table.cell(row, col).text.replace(key, value)


                    # Now, we suppress the tags TagStartGeneralQuestion and TagEndGeneralQuestion from the questions
                    for docpara in document.paragraphs:
                        if TagStartGeneralQuestion in docpara.text:
                            docpara.text = docpara.text.replace(TagStartGeneralQuestion, "")
                        if TagEndGeneralQuestion in docpara.text:
                            docpara.text = docpara.text.replace(TagEndGeneralQuestion, "")

                    # We create a new version of the document with the answers
                    document.save(PathForOutputsAndLogs+ r'/' + NameOfDocument[:-4] + "_with_answers.docx")
                except IOError:
                        MessageError = str(datetime.now()) + ' Error encountered when opening for writing the Word docx file ' + file
                        logging.error(MessageError)
                        print(MessageError)
                finally:        
                    f.close()

    print('End of the write program')
    return
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF "WRITE ANSWERS INTO NEW AAP" FUNCTION @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Settings for the path files
Path_where_we_put_Outputs = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/Pour les logs/' 
Folder_where_the_files_are = r'/Users/jfm/Library/CloudStorage/OneDrive-Personnel/Python yc Dev D4G/3 - Dev IA Asso/LesFilesA Lire/'
from docx import Document # import de python-docx

#tuple(c.text for c in r.cells) for r in table.rows


# Read the questions in the files and put them into a dictionnary
The_Dict_Of_Questions = Read_Questions_From_docx (Folder_where_the_files_are, Path_where_we_put_Outputs)

# TO DO : The main programm should then call the AI to answer the questions of the dictionary "The_Dict_Of_Questions"
# and put the answers into a "dictionnary of answers" with the same keys (key of question = key of answer)

# For the moment, we create a dictionary of answers with the same keys as the dictionary of questions
# by just taking the question as the answer we just put "ANSWER TO: " + the question

for key, value in The_Dict_Of_Questions.items():
        The_Dict_Of_Answers = {key:  value for key,  value in The_Dict_Of_Questions.items()}
for key, value in The_Dict_Of_Answers.items():
        The_Dict_Of_Answers[key] = ' ANSWER TO: ' + value
# Write the answers into the docx files just below the questions
Write_Answers_in_docx (Path_where_we_put_Outputs, The_Dict_Of_Answers, Path_where_we_put_Outputs)

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END OF MAIN PROGRAM @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

Type 0 table
in row = 0 and Col = 0, the content is Résumé synthétique du projet (5 lignes maximum)
in row = 1 and Col = 0, the content is Exemple Docx de Questions.docx - 57ff14bd15b04af69a5fe8ab3a7c7e6a
in row = 2 and Col = 0, the content is  
in row = 3 and Col = 0, the content is Résumé NON synthétique du projet (200 lignes minimum)
in row = 4 and Col = 0, the content is Exemple Docx de Questions.docx - f1587f324d204c5f931ff3a519b5c53b
in row = 5 and Col = 0, the content is  
Nbr of columns = 1 and Nbr of rows =  6
in row = 0 and Col = 0, the content is Résumé synthétique du projet (5 lignes maximum)
in row = 1 and Col = 0, the content is Exemple Docx de Questions.docx - 57ff14bd15b04af69a5fe8ab3a7c7e6a
in row = 2 and Col = 0, the content is  
in row = 3 and Col = 0, the content is Résumé NON synthétique du projet (200 lignes minimum)
in row = 4 and Col = 0, the content is Exemple Docx de Questions.docx - f1587f324d204c5f931ff3a519b5c53b
in row = 5 and Col = 0, the content is  

Ty

## (Optional in the beginning) Chunk and embedd documents

Chunking and embedding documents is a way to implement a RAG (Retrieval Augmented Generation). 

To learn about this concept, you can check the following links :

Here are also useful resources to implement a RAG in python using langchain :



!! It is important to note that while RAG is a common way to provide LLMs with context, specific methods can be used for this project. For instance, maybe that all documents have an "information about x" section that can be directly retrieved with regex methods to provide the model with.

For regex methods, you can find documentation here :


In [10]:
# Here split the document into chunks

In [11]:
# Here embed those chunks

In [12]:
# (Optional) Here you can store those embedded chunks into a vector store

## call a large language model via an API (e.g. Mistral API call - use free tiers)

Here we're gonna call a model (and pass him the context if already implemented before)

Some links you can check to learn more if you don't know how it works :

Langchain (one of the classic tools for this kind of task)


<b>To run a model locally</b>

With Ollama :

With huggingface : 

In [13]:
"""
Here, first write your credentials for API call (don't push it on git !! Use environment variables)
or load the model in the notebook kernel if you want to use a model locally
"""

"\nHere, first write your credentials for API call (don't push it on git !! Use environment variables)\nor load the model in the notebook kernel if you want to use a model locally\n"

In [14]:
"""
Then, implement API calling (langchain chain + prompt engineering)
You can divide the whole process in several sub-questions if the model can't take enough context at once,
or if it does not perform well enough.
"""

"\nThen, implement API calling (langchain chain + prompt engineering)\nYou can divide the whole process in several sub-questions if the model can't take enough context at once,\nor if it does not perform well enough.\n"

## (Very very optional) Implement a langgraph to enhance generation performances with agentic behavior

This step should not be necessary but once everything else is set up, you can play with it.

Documentation : 

In [15]:
# Langgraph implementation